In [ ]:
import grpc
import swoq_pb2
import swoq_pb2_grpc
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output, display

In [ ]:
player_id = '6616b1c5bd0a697480a68319'

In [ ]:
colors = {
    0: [  0,   0,   0], # UNKNOWN
    1: [ 50,  50,  50], # EMPTY
    2: [  0,   0, 255], # PLAYER
    3: [147, 124,  93], # WALL
    4: [230, 217, 177], # EXIT

    5: [192,   0,   0], # DOOR_RED
    6: [255,   0,   0], # KEY_RED
}

In [ ]:
def get_map_image(map, height, width):
    map_img = np.zeros((height, width, 3), dtype=np.float32)
    i = 0
    for y in range(height):
        for x in range(width):
            map_img[y, x] = (np.array(colors[map[i]]) / 255).astype(np.float32)
            i += 1
    return map_img

def plot_map(map, height, width):
    map_img = get_map_image(map, height, width)
    fig = plt.figure()
    plt.gca().set_axis_off()
    plt.tight_layout(pad=0)
    img = plt.imshow(map_img)
    plt.show()
    return (fig, img)

def update_map(frame, map, height, width):
    map_img = get_map_image(map, height, width)

    fig, img = frame
    img.set_data(map_img)
    clear_output(wait=True)
    display(fig)

In [ ]:
with grpc.insecure_channel('localhost:5009') as channel:
    stub = swoq_pb2_grpc.TrainingStub(channel)
    startResponse = stub.StartGame(swoq_pb2.StartRequest(playerId=player_id))

print(f'{startResponse.result=}')

game_id = startResponse.gameId
print(f'{game_id=}')
height = startResponse.height
width = startResponse.width
print(f'{height=}, {width=}')

map = np.array(startResponse.state.map)
plot_map(map, height, width)


In [ ]:
def softmax(a):
    e = np.exp(a)
    return e / np.sum(e)

def pick_random_move(map):
    player_pos = np.unravel_index(np.argwhere(map == 2).flatten()[0], (height, width))
    target_pos = np.array([height-2, width-1])
    print(f'{player_pos=}, {target_pos=}')

    diff = target_pos - player_pos
    direction = np.sign(diff)

    # increase probability for actions in right direction
    moves = np.ones((4,))
    if direction[0] < 0: moves[0] += 1 # NORTH
    if direction[0] > 0: moves[1] += 1 # SOUTH
    if direction[1] < 0: moves[2] += 1 # WEST
    if direction[1] > 0: moves[3] += 1 # EAST
    print(f'{moves=}')
    moves = softmax(moves)

    # choose random move
    directions = [swoq_pb2.NORTH, swoq_pb2.SOUTH, swoq_pb2.WEST, swoq_pb2.EAST]
    direction = np.random.choice(directions, p=moves)
    print(f'{direction=}')
    return direction

In [ ]:
def move(direction):
    global frame
    with grpc.insecure_channel('localhost:5009') as channel:
        stub = swoq_pb2_grpc.TrainingStub(channel)
        moveResponse = stub.Move(swoq_pb2.MoveRequest(gameId=game_id, direction=direction))

    global map
    map = np.array(moveResponse.state.map)
    update_map(frame, map, height, width)
    print(f'{moveResponse.result=}')
    print(f'{moveResponse.state.finished=}')
    print(f'{moveResponse.state.inventory=}')

In [ ]:
sequence = 'SSSSSEEENNSSSSESESESESESESESESESE'

frame = plot_map(map, height, width)

for s in sequence:
    match s:
        case 'N': move(swoq_pb2.NORTH)
        case 'E': move(swoq_pb2.EAST)
        case 'S': move(swoq_pb2.SOUTH)
        case 'W': move(swoq_pb2.WEST)